In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned/cleaned_dataset.csv


In [2]:
!pip install pyts


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from pyts.image import MarkovTransitionField
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import warnings
from collections import Counter
import gc
from tensorflow.keras import backend as K

warnings.filterwarnings("ignore")

# Load dataset
df = pd.read_csv('/kaggle/input/cleaned/cleaned_dataset.csv')  # Replace with your path
df['DateTime'] = pd.to_datetime(df['DateTime'])
df = df.sort_values(['SubjectID', 'DateTime'])

# Preprocess features
feature_cols = ['AccX', 'AccY', 'AccZ', 'Temp', 'EDA', 'HeartRate', 'SpO2']
target_col = 'Label'

scaler = StandardScaler()
df[feature_cols] = scaler.fit_transform(df[feature_cols])

def create_windows(df, feature_cols, target_col, window_size=100, step=20):
    X = []
    y = []
    for subject_id, subject_df in df.groupby('SubjectID'):
        subject_data = subject_df.reset_index(drop=True)
        features = subject_data[feature_cols].values
        labels = subject_data[target_col].values
        for start in range(0, len(features) - window_size, step):
            end = start + window_size
            window_x = features[start:end]
            window_y = labels[start:end]
            label = Counter(window_y).most_common(1)[0][0]
            X.append(window_x)
            y.append(label)
    return np.array(X), np.array(y)

X, y = create_windows(df, feature_cols, target_col)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Convert time-series to images using MTF
def time_series_to_image_mtf(sample, image_size=64):
    mtf = MarkovTransitionField(image_size=image_size)
    img_list = []
    for i in range(sample.shape[1]):
        ts = sample[:, i]
        img = mtf.fit_transform(ts.reshape(1, -1))[0]
        img_list.append(img)
    img = np.stack(img_list, axis=-1)
    return img

image_data = np.array([time_series_to_image_mtf(seq) for seq in X])

# CNN model
def build_image_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Evaluation
def evaluate_model(model, X_test, y_test, label_encoder, title="Model"):
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

    cm = confusion_matrix(y_true_classes, y_pred_classes)
    specificity_per_class = []
    for i in range(len(cm)):
        tn = np.sum(np.delete(np.delete(cm, i, axis=0), i, axis=1))
        fp = np.sum(np.delete(cm, i, axis=0)[:, i])
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        specificity_per_class.append(specificity)
    weighted_specificity = np.average(specificity_per_class, weights=np.bincount(y_true_classes))

    print(f"\n{title}")
    print(f"Accuracy     : {accuracy:.4f}")
    print(f"Precision    : {precision:.4f}")
    print(f"Recall       : {recall:.4f}")
    print(f"F1 Score     : {f1:.4f}")
    print(f"Specificity  : {weighted_specificity:.4f}")

    return accuracy, precision, recall, f1, weighted_specificity

# Intra-Subject Classification: Ensure data consistency for splitting
results_subjectwise = {}
for subject_id, subject_df in df.groupby('SubjectID'):
    X_subject, y_subject = create_windows(subject_df, feature_cols, target_col)
    
    # Convert time-series data to images for subject-wise classification
    image_data_subject = np.array([time_series_to_image_mtf(seq) for seq in X_subject])
    
    # Ensure labels and features have the same length for splitting
    y_subject_encoded = le.transform(y_subject)
    
    X_train_subj, X_test_subj, y_train_subj, y_test_subj = train_test_split(
        image_data_subject, to_categorical(y_subject_encoded), test_size=0.2, stratify=y_subject_encoded, random_state=42
    )
    
    img_model_subj = build_image_model(X_train_subj.shape[1:], y_cat.shape[1])
    img_model_subj.fit(X_train_subj, y_train_subj, validation_data=(X_test_subj, y_test_subj),
                       epochs=20, batch_size=32)
    
    # Evaluate
    acc, precision, recall, f1, specificity = evaluate_model(img_model_subj, X_test_subj, y_test_subj, le, title=f"Subject {subject_id}")
    results_subjectwise[subject_id] = {'Accuracy': acc, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Specificity': specificity}
    
    # Save model
    model_filename = f"/kaggle/working/subject_{subject_id}_cnn_model.h5"
    img_model_subj.save(model_filename)
    print(f"Model for Subject {subject_id} saved as {model_filename}")
# === Summary ===

print("\n=== Summary Metrics ===")

# Average Per-Subject Metrics
print("\n=== Per-Subject Metrics ===")
print("Average Per-Subject Accuracy:", np.mean([result['Accuracy'] for result in results_subjectwise.values()]))
print("Average Per-Subject Precision:", np.mean([result['Precision'] for result in results_subjectwise.values()]))
print("Average Per-Subject Recall:", np.mean([result['Recall'] for result in results_subjectwise.values()]))
print("Average Per-Subject F1 Score:", np.mean([result['F1'] for result in results_subjectwise.values()]))
print("Average Per-Subject Specificity:", np.mean([result['Specificity'] for result in results_subjectwise.values()]))

# Save Intra-Subject Results
with open("intra_subject_results.txt", "w") as f:
    f.write("=== Intra-Subject Evaluation Results ===\n\n")
    for subject_id, metrics in results_subjectwise.items():
        f.write(f"Subject {subject_id}:\n")
        for metric_name, metric_value in metrics.items():
            f.write(f"  {metric_name}: {metric_value:.4f}\n")
        f.write("\n")

    f.write("=== Average Intra-Subject Metrics ===\n")
    f.write(f"Accuracy     : {np.mean([res['Accuracy'] for res in results_subjectwise.values()]):.4f}\n")
    f.write(f"Precision    : {np.mean([res['Precision'] for res in results_subjectwise.values()]):.4f}\n")
    f.write(f"Recall       : {np.mean([res['Recall'] for res in results_subjectwise.values()]):.4f}\n")
    f.write(f"F1 Score     : {np.mean([res['F1'] for res in results_subjectwise.values()]):.4f}\n")
    f.write(f"Specificity  : {np.mean([res['Specificity'] for res in results_subjectwise.values()]):.4f}\n")




2025-04-14 11:46:35.882349: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744631196.140358      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744631196.223422      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 11:48:56.947597: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 173ms/step - accuracy: 0.4926 - loss: 1.2178 - val_accuracy: 0.5381 - val_loss: 0.9942
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 183ms/step - accuracy: 0.6205 - loss: 0.9363 - val_accuracy: 0.6853 - val_loss: 0.8528
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - accuracy: 0.6708 - loss: 0.8437 - val_accuracy: 0.7310 - val_loss: 0.7720
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 163ms/step - accuracy: 0.6991 - loss: 0.8076 - val_accuracy: 0.6954 - val_loss: 0.7088
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 164ms/step - accuracy: 0.7093 - loss: 0.7296 - val_accuracy: 0.7766 - val_loss: 0.6235
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 161ms/step - accuracy: 0.7094 - loss: 0.6818 - val_accuracy: 0.7259 - val_loss: 0.6124
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 160ms/step - accuracy: 0.7500 - loss: 0.6086 - val_accuracy: 0.8173 - val_loss: 0.5759
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 161ms/step - accuracy: 0.7400 - loss: 0.6138 - val_accuracy: 0.